In [2]:
import numpy as np
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [109]:
img_id = "CA184632V0340"

In [110]:
import src.load.load_image as ld

#path_data_fld = "C:/Users/Felix/Documents/GitHub/Antarctic_TMA/data"
path_data_fld = "/data_1/ATM/data_1/aerial/TMA/downloaded"

# load image to geo-reference
image = ld.load_image(path_data_fld + f"/{img_id}.tif")

print(image.shape)

(9522, 10140)


In [111]:
import src.base.connect_to_database as ctd

sql_string = f"SELECT * FROM images WHERE image_id='{img_id}'"
conn = ctd.establish_connection()
data = ctd.execute_sql(sql_string, conn)

azimuth = data.iloc[0]['azimuth']
month = data.iloc[0]['date_month']

azimuth = 360 - azimuth + 90

print(azimuth, month)

sql_string_approx = f"SELECT ST_AsText(footprint_approx) AS footprint_approx FROM images_extracted WHERE image_id='{img_id}'"
data_approx = ctd.execute_sql(sql_string_approx, conn)
footprint_approx = data_approx.iloc[0]['footprint_approx']
print(footprint_approx)

338.8565 12
POLYGON((-1922134.7450523146 748024.2434102389,-1925221.0565897613 740043.9250523145,-1933201.3749476855 743130.2365897611,-1930115.0634102388 751110.5549476855,-1922134.7450523146 748024.2434102389))


In [112]:
mask = np.ones_like(image)
pad_width = 500

mask[:pad_width, :] = 0  # Top edge
mask[-pad_width:, :] = 0  # Bottom edge
mask[:, :pad_width] = 0  # Left edge
mask[:, -pad_width:] = 0  # Right edge

In [114]:
import src.georef.georef_sat as gs

georefSat = gs.GeorefSatellite(min_tps_final=25,
                               enhance_image=True, locate_image=True, tweak_image=True,filter_outliers=True)

transform, residuals, tps, conf =  georefSat.georeference(image, footprint_approx, mask, azimuth, month)


Geo-reference image by satellite
Adjusted image resolution with zoom-factor (0.0884, 0.0859)
Adjusted image resolution with zoom-factor (0.0884, 0.0859)
Locate image position
  Check tile 1 (Coords: [0, -7386], Order 1)
  0 points were found in this tile.
  Check tile 2 (Coords: [0, 7386], Order 1)
  21 points were found in this tile.
  Check tile 3 (Coords: [0, -14772], Order 2)
  0 points were found in this tile.
  Check tile 4 (Coords: [0, 14772], Order 2)
  0 points were found in this tile.
Best tile is [0, 7386] with 21 tie-points (0.226)
Tweak image position
  Tweak image (1/10) with (773, -59)
  7 points found in tweak 1 of 10
  Points going down (7 < 21)
  Tweak image (2/10) with (-1250, -892)
  14 points found in tweak 2 of 10
  Points going down (14 < 21)
  Break the loop
Tweaking finished with 21 tie-points
9 outliers removed with RANSAC


In [115]:
import src.georef.apply_transform as af
import src.export.export_shape as es

af.apply_transform(image, transform, f"/data_1/ATM/temp/{img_id}_georeferenced.tif")